In [ ]:
import random
import csv
import logging

from shop_crawler import *
from selenium_helper import *
import common_actors

from contextlib import contextmanager

### Prepare different urls to analyze

In [ ]:
# All urls
all_urls = []
with open('../resources/pvio_vio_us_ca_uk_sample1.csv', 'r') as f:
    rows = csv.reader(f)
    for row in rows:
        url = row[0]
        if url:
            all_urls.append(url)

# Random sample urls
random.seed(1)
sample_urls = random.sample(all_urls, 100)

# Some good urls to analyze by hands
good_urls = [
    'curlebotanicals.com',
    'theglamourshop.com',
    'vape-fuel.com',
    'firstfitness.com',
    'sandlakedermatology.com',
    'getwaave.com',
    'dixieems.com',
    'jonessurgical.com',
    'srandd.com',
    'ambarygardens.com',
    'anabolicwarfare.com'
]


### Mock user and payment information

In [ ]:
user_info = UserInfo(
    first_name = 'John',
    last_name = 'Smith',
    country = 'United States',
    home = 34,
    street = 'Ocean drive',
    city = 'Miami',
    zip = '33125',
    state = 'FLorida',
    
    phone = '1231232',
    email = 'john@service.com'
)

billing_info = PaymentInfo(
    card_number = '1413232312312321',
    expire_date_year = 2020,
    expire_date_month = 12,
    cvc = '123'
)

In [ ]:
selenium_path = '/home/aleksei/dist/selenium/chromedriver'

@contextmanager
def get_crawler(headless=True):
    global user_info, billing_info, selinium_path
    crawler = ShopCrawler(user_info, billing_info, selenium_path, headless=headless)
    common_actors.add_crawler_extensions(crawler)
    
    yield crawler

def get_driver(headless=True):
    global selenium_path   
    return create_chrome_driver(selenium_path, headless=headless)


### Set up logging level

In [ ]:
logger = logging.getLogger('shop_crawler')
logger.setLevel(logging.DEBUG)

handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

### Start crawling for every url

In [ ]:
results = []
with get_crawler(headless=False) as crawler:
    for url in good_urls:
        print('\n\n{}'.format(url))
        status = crawler.crawl(url, attempts=1)
        print(status)
        results.append(status)


In [ ]:
states = {}
for status in results:
    if isinstance(status, ProcessingStatus):
        states[status.state] = states.get(status.state, 0) + 1
        
print(states)

### Test Empty cart processing

In [ ]:
# Make it True if you want to test empty cart detection
test_empty_cart = False

checkout_pages = [
    'http://vape-fuel.com/checkout/cart/',
    'https://store-bb7f1.mybigcommerce.com/cart.php',
    'https://www.firstfitness.com/index.php?pg=signup',
    'https://store.sandlakedermatology.com/checkout/cart/',
    'https://www.dixieems.com/cart.asp',
    'http://www.jonessurgical.com/quick-order'    
]

if test_empty_cart:
    driver = get_driver()
    
    for url in checkout_pages:
        driver.get(url)
        
        is_empty_cart = common_actors.is_empty_cart(driver)
        print('url: {}, is empty cart: {}'.format(url, is_empty_cart))
        assert is_empty_cart

    driver.quit()

In [ ]:
test_broken_sites = False

not_available = [
    'juicyliquid.com',
    'gotbloody.com',
    'trelexa.org'
]

error = [
    'tampahumidor.com',
    'seriouscigars.com'    
]

timeout = [
    'moneynetint.com'
]

domains_for_sale = [
    'walkinmycloset.com'
]
        

if test_broken_sites:
    with get_crawler() as crawler:
        for url in domains_for_sale:
            print(url)
            status = crawler.crawl(url)
            assert isinstance(status, NotAvailable), '{} must be not available'.format(url)
        
        for url in timeout:
            print(url)
            status = crawler.crawl(url)
            assert isinstance(status, Timeout) or isinstance(status, NotAvailable), \
                '{} must be not timed out'.format(url)

        for url in not_available:
            print(url)
            status = crawler.crawl(url)
            assert isinstance(status, NotAvailable), '{} must be not available'.format(url)

        for url in error:
            print(url)
            status = crawler.crawl(url)
            assert isinstance(status, RequestError), '{} must produce an error'.format(url)
        
        